# Installing Z3 and Imports

In [1]:
'''
!pip install z3-solver
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install anchor-exp
'''

'\n!pip install z3-solver\n!pip install pandas\n!pip install numpy\n!pip install sklearn\n!pip install anchor-exp\n'

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from z3 import *
import time

# Training Linear and Polynomial SVMs

## Data Preprocessing.

In [3]:
wine = datasets.load_wine()

In [4]:
df = pd.DataFrame(wine.data, columns = wine.feature_names)

In [5]:
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [6]:
normalized_df=(wine.data-wine.data.min())/(wine.data.max()-wine.data.min())
#normalized_df=(cancer.data-cancer.data.mean())/cancer.data.std()

In [7]:
wine.data

array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [8]:
normalized_df.min(),normalized_df.max()

(0.0, 1.0)

In [9]:
np.where(wine.target == wine.target[0],0,1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [10]:
def check_targets(original_set):
    original_unique = np.unique(original_set)
    print("Original Targets: ",original_unique,"\nDesired Targets: [-1,1]")
    print("Is original the desired [1,-1]? ", np.array_equiv(original_unique,np.array([-1,1])))
    if not np.array_equiv(original_unique,np.array([-1,1])):
        if 1 in original_unique:
            print("1 exists in dataset")
            new = np.select([original_set == original_unique[0]],[-1],original_set)
        elif -1 in original_unique:
            print("-1 exists in dataset")
            new = np.select([original_set == original_unique[1]],[1],original_set)
        else:
            print("Neither exists in dataset")
            new = np.select([original_set == original_unique[0],original_set == original_unique[1]],[-1,1],original_set)
        #indexes = original_set[np.where(original_set == unique_elems[0])]
        print("New datasets consists of: ",np.unique(new))
        return new

In [11]:
targets = check_targets(np.where(wine.target == wine.target[0],0,1))

Original Targets:  [0 1] 
Desired Targets: [-1,1]
Is original the desired [1,-1]?  False
1 exists in dataset
New datasets consists of:  [-1  1]


## Data Separation and Training

In [12]:
X_train, X_test, y_train, y_test = train_test_split(normalized_df, targets, test_size=0.3,random_state=107) # 70% training and 30% test

In [13]:
len(X_train[0])

13

In [14]:
def create_linear_classifier(kernel_type='linear'):
    return svm.SVC(kernel=kernel_type)
def create_poly_classifier(kernel_type='poly',my_degree=2,my_gamma=1/30):
    return svm.SVC(kernel=kernel_type, degree = my_degree,gamma=my_gamma)

In [15]:
clf = create_linear_classifier()
#poly = create_poly_classifier('poly',2,1/(X_train.var() * len(X_train[0])))

#Train the models using the training sets
clf.fit(X_train, y_train)
#poly.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
#poly_y_pred = poly.predict(X_test)
print("Accuracy Linear:", metrics.accuracy_score(y_test, y_pred))
#print("Accuracy Poly:", metrics.accuracy_score(y_test, poly_y_pred))

Accuracy Linear: 0.8888888888888888


In [16]:
y_pred_train = clf.predict(X_train)
#poly_pred_train = poly.predict(X_train)
print("Accuracy on Training:", metrics.accuracy_score(y_train, y_pred_train))
#print("Accuracy on Training:", metrics.accuracy_score(y_train, poly_pred_train))

Accuracy on Training: 0.9354838709677419


## SVM Decision Function For The First Element of Training Dataset

In [17]:
#Sum (coef @ sup_vec @ X[index] + bias)
((clf.dual_coef_ @ clf.support_vectors_) @ X_train[0].reshape(1, len(X_train[0])).T + clf.intercept_)[0][0]

-1.6118815066885395

In [18]:
#(poly.dual_coef_ @ (((poly.support_vectors_ @ X_train[0].reshape(1, len(X_train[0])).T) * poly.gamma + poly.coef0) ** poly.degree) + poly.intercept_)[0][0]

In [19]:
#Linear SVM Decision Function
print("DualCoef / Support Vectors / X_Train.T Reshaped / Intercept (bias)")
clf.dual_coef_.shape, clf.support_vectors_.shape, X_train[0].reshape(1, len(X_train[0])).T.shape, clf.intercept_.shape

DualCoef / Support Vectors / X_Train.T Reshaped / Intercept (bias)


((1, 58), (58, 13), (13, 1), (1,))

In [20]:
#Polynomial SVM Decision Function
#print("DualCoef / Support Vectors / X_Train.T Reshaped / Gamma / Coef0 / Degree / Intercept (bias)")
#poly.dual_coef_.shape, poly.support_vectors_.shape, X_train[0].reshape(1, len(X_train[0])).T.shape,poly.gamma, poly.coef0, poly.degree, poly.intercept_

# Defining Thresholds and Finding Rejecteds

In [21]:
def limits(classifier,data):
    dec_fun = classifier.decision_function(data)
    lim_pos = dec_fun[np.argmax(dec_fun)]
    lim_neg = dec_fun[np.argmin(dec_fun)]
    return dec_fun, lim_pos, lim_neg

In [63]:
def find_thresholds(decfun,t1,t2,wr,chosen_min='E'):
    solution = {'WR':0,'T1':0, 'T2':0,'E':0,'R':0,'EWRR':0}
    index = None
    n_elements = decfun.shape[0]
    for i,wr_ in enumerate(wr):
      for j in range(0,len(t1)):
        #Get Number of Rejected
        positive_indexes = np.where(decfun >= t1[j])
        negative_indexes = np.where(decfun < t2[j])
        rejected_indexes = np.where((decfun < t1[j]) & (decfun >= t2[j]))
        R = rejected_indexes[0].shape[0]
        #np.array(positive_indexes).shape,np.array(negative_indexes).shape, R

        #Get Number of Misclassifications
        class_p = y_train[positive_indexes]
        class_n = y_train[negative_indexes]
        error_p = np.where(class_p == np.unique(y_train)[0])[0].shape[0]
        error_n = np.where(class_n == np.unique(y_train)[1])[0].shape[0]
        E = (error_p + error_n) - R
        #print("T1 ",round(t1[j],4),"T2 ",round(t2[j],4),"E",round(E,4),"Rej",R,"Pos ", positive_indexes[0].shape[0], "Neg ",negative_indexes[0].shape[0])
        if chosen_min=='R':
            if (i == 0 and i == j) or R < solution['R']:
                solution['WR'] = wr_
                solution['T1'] = t1[j]
                solution['T2'] = t2[j]
                solution['E'] = E
                solution['R'] = R
                solution['EWRR'] = E + wr_ * R
        elif chosen_min=='E':
            if (i == 0 and i == j) or E < solution['E']:
                solution['WR'] = wr_
                solution['T1'] = t1[j]
                solution['T2'] = t2[j]
                solution['E'] = E
                solution['R'] = R
                solution['EWRR'] = E + wr_ * R
        elif chosen_min=='EWRR':
            if (i == 0 and i == j) or (E + wr_ * R) < solution['EWRR']:
                solution['WR'] = wr_
                solution['T1'] = t1[j]
                solution['T2'] = t2[j]
                solution['E'] = E
                solution['R'] = R
                solution['EWRR'] = E + wr_ * R
        else:
            return 'Chosen option "' +chosen_min+'" is invalid'
    print('Thresholds by min(',chosen_min,') from solution: ',solution)
    return solution['T1'], solution['T2']      
                

In [23]:
def find_indexes(decfun,t1,t2):
    positive_indexes = np.where(decfun >= t1)[0]
    negative_indexes = np.where(decfun < t2)[0]
    rejected_indexes = np.where((decfun < t1) & (decfun >= t2))[0]
    R = rejected_indexes.shape[0]
    return positive_indexes,negative_indexes,rejected_indexes

In [24]:
def find_thresholds_and_indexes(classifier,data,wr = None):  
    dec_fun,lim_pos,lim_neg = limits(classifier,data)
    print("Superior Limit: ",lim_pos,"\nInferior Limit: ",lim_neg)
    if wr == None:
        wr = [0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.36, 0.4, 0.44, 0.48]
    t1 = []
    t2 = []
    for i in range (1,21):
      t1.append(0.05*i*lim_pos)
      t2.append(0.05*i*lim_neg)  
    T1,T2 = find_thresholds(dec_fun,t1,t2,wr)
    pos_idx,neg_idx,rej_idx = find_indexes(dec_fun,T1,T2)
    return T1, T2, pos_idx, neg_idx, rej_idx,lim_pos,lim_neg

# Implementing SVM function for Z3 Solver

## Z3 Decision Function Elements

In [25]:
np.RealVal = np.vectorize(RealVal) 
np.RealVector = np.vectorize(RealVector) 

In [26]:
def to_z3_conversion(classifier,training_set):
    z3_dual_coef = np.RealVal(classifier.dual_coef_)
    z3_support_vectors = np.RealVal(classifier.support_vectors_)
    z3_intercept_ = np.RealVal(classifier.intercept_)
    z3_X_Train = np.RealVector('x',training_set.shape[1])
    if classifier.kernel == 'poly':
        z3_gamma = np.RealVal(classifier.gamma)
        z3_coef0 = np.RealVal(classifier.coef0)
        z3_degree = np.RealVal(classifier.degree)
        return z3_dual_coef,z3_support_vectors,z3_intercept_,z3_X_Train, z3_gamma,z3_coef0,z3_degree
    return z3_dual_coef,z3_support_vectors,z3_intercept_,z3_X_Train

# Z3 with Reject Option

## Explaining the Classifier's Decision Function and Finding Relevant Features

In [27]:
def z3_explanation(classifier,t1, t2, X, z3_coef, z3_sup_vec, z3_X, z3_intercept, reject_indexes,sup_lim,inf_lim,
                   z3_gamma = None, z3_coef0 = None, z3_degree = None, show_values=True, min=0,max=1,positive=False,negative=False,rejected=False):
    elapsed_time = []
    relevant = []
    irrelevant = []
    global_values = []
    print("Number of Instances: ", len(reject_indexes))
    solver = Solver()
    if classifier.kernel=='linear':
        print("Classifier: Linear")
        if rejected:
            print("Declared: Rejected Instances")
            solver.add(Or(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t1,
                          ((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t2))
        elif positive:
            print("Declared: Positive Instances")
            solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t1)
        elif negative:
            print("Declared: Negative Instances")
            solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t2)
        else:
            print("WARNING: Must declare if are positive,negative or rejected instances!")
        solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= inf_lim)
        solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] <= sup_lim)
    elif classifier.kernel=='poly':
        print("Classifier: Polynomial")
        solver.add(Or(((z3_dual_coef @ (((z3_support_vectors @ z3_X.reshape(1, len(z3_X)).T) * z3_gamma + z3_coef0) ** z3_degree) + z3_intercept_)[0][0] >= t1),
                      (z3_dual_coef @ (((z3_support_vectors @ z3_X.reshape(1, len(z3_X)).T) * z3_gamma + z3_coef0) ** z3_degree) + z3_intercept_)[0][0] < t2))
    for j in range(0, len(z3_X)):
        solver.add(z3_X[j] >= min)
        solver.add(z3_X[j] <= max)
    solver.push()
    for i in range(0, len(reject_indexes)):
        # Add Assertions for 0<= feature <= 1
        index_list = list(range(len(z3_X)))
        unsat_list = []
        sat_list = []
        values = []

        # Select a feature and unfix it
        start = time.perf_counter()
        for z in range(0, len(z3_X)):   
            for j in range(0, len(z3_X)):
                if j != z and j in index_list:  # Choose one to check influence
                    solver.add(z3_X[j] == X[reject_indexes[i]][j])
            
            check = solver.check()
            if check == sat:
                model = solver.model()
                value = model[z3_X[z]].numerator_as_long() / model[z3_X[z]].denominator_as_long()
                sat_list.append(z)
                values.append(value)
                if show_values:
                    print('i = ', i, z, check, X[reject_indexes[i]][z], value)
            else:
                unsat_list.append(z)
                index_list.remove(z)
                if show_values:
                    print('i = ', i, z, check)
            
            solver.pop()
            solver.push()
        elapsed_time.append(time.perf_counter()  - start)    
        print("Finished ", i)
        relevant.append(sat_list)
        irrelevant.append(unsat_list)
        global_values.append(values)
        # print("Relevant: ",sat_list, '\nUnsat List: ',unsat_list,'\n')      
    for i in range(0, len(relevant)):
        if (show_values):
            print('Instance ', i, '\nRelevant Features: ', relevant[i], '\nValues: ', global_values[i], '\nIrrelevant Features: ',
                  irrelevant[i], '\nElapsed time: ',elapsed_time[i],'seconds\n\n')
    
    print("Tamanho médio de explicação: ",sum(len(x) for x in relevant)/len(relevant)," - Custo médio: ",round(sum(elapsed_time)/len(elapsed_time),5),"seg(s)")        
    return relevant, irrelevant, elapsed_time

### For Linear Classifier

#### Get thresholds and the rejected for Linear

In [64]:
T1,T2, positive_indexes,negative_indexes,rejected_indexes,lim_pos,lim_neg = find_thresholds_and_indexes(clf,X_train)
T1,T2, positive_indexes.shape[0],negative_indexes.shape[0],rejected_indexes.shape[0]

Superior Limit:  2.02334139162129 
Inferior Limit:  -1.8159002884523145
Thresholds by min( E ) from solution:  {'WR': 0.04, 'T1': 0.9105036262295805, 'T2': -0.8171551298035415, 'E': 0.0, 'R': 47, 'EWRR': 1.8800000000000001}


(0.9105036262295805, -0.8171551298035415, 60, 17, 47)

#### Get Z3's equivalent to linear classifier's decision function

In [29]:
z3_dual_coef,z3_support_vectors,z3_intercept_,z3_X_Train = to_z3_conversion(clf,X_train)

In [30]:
rejected_linear_relevant = []
rejected_linear_irrelevant = []
positive_linear_relevant = []
positive_linear_irrelevant = []
negative_linear_relevant = []
negative_linear_irrelevant = []
rejected_elapsed_time = []
positive_elapsed_time = []
negative_elapsed_time = []

In [31]:
if len(rejected_indexes)!=0:
    if len(rejected_indexes)>=50:
        rejected_linear_relevant, rejected_linear_irrelevant, rejected_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,rejected_indexes[0:50],
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=False, rejected = True)
    else:
        rejected_linear_relevant, rejected_linear_irrelevant, rejected_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,rejected_indexes,
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=False, rejected = True)

Number of Instances:  2
Classifier: Linear
Declared: Rejected Instances
Finished  0
Finished  1
Tamanho médio de explicação:  5.5  - Custo médio:  0.02102 seg(s)


In [32]:
if len(positive_indexes)!=0:
    if len(positive_indexes)>=50:
        positive_linear_relevant, positive_linear_irrelevant, positive_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,positive_indexes[0:50],
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=False, positive = True)
    else:
        positive_linear_relevant, positive_linear_irrelevant, positive_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,positive_indexes,
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=False, positive = True)

Number of Instances:  50
Classifier: Linear
Declared: Positive Instances
Finished  0
Finished  1
Finished  2
Finished  3
Finished  4
Finished  5
Finished  6
Finished  7
Finished  8
Finished  9
Finished  10
Finished  11
Finished  12
Finished  13
Finished  14
Finished  15
Finished  16
Finished  17
Finished  18
Finished  19
Finished  20
Finished  21
Finished  22
Finished  23
Finished  24
Finished  25
Finished  26
Finished  27
Finished  28
Finished  29
Finished  30
Finished  31
Finished  32
Finished  33
Finished  34
Finished  35
Finished  36
Finished  37
Finished  38
Finished  39
Finished  40
Finished  41
Finished  42
Finished  43
Finished  44
Finished  45
Finished  46
Finished  47
Finished  48
Finished  49
Tamanho médio de explicação:  1.4  - Custo médio:  0.01113 seg(s)


In [33]:
if len(negative_indexes)!=0:
    if len(negative_indexes)>=50:
        negative_linear_relevant, negative_linear_irrelevant, negative_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,negative_indexes[0:50],
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=False, negative = True)
    else:
        negative_linear_relevant, negative_linear_irrelevant, negative_elapsed_time = z3_explanation(clf,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,negative_indexes,
                                                                          sup_lim=lim_pos,inf_lim=lim_neg,show_values=False, negative = True)

Number of Instances:  32
Classifier: Linear
Declared: Negative Instances
Finished  0
Finished  1
Finished  2
Finished  3
Finished  4
Finished  5
Finished  6
Finished  7
Finished  8
Finished  9
Finished  10
Finished  11
Finished  12
Finished  13
Finished  14
Finished  15
Finished  16
Finished  17
Finished  18
Finished  19
Finished  20
Finished  21
Finished  22
Finished  23
Finished  24
Finished  25
Finished  26
Finished  27
Finished  28
Finished  29
Finished  30
Finished  31
Tamanho médio de explicação:  1.125  - Custo médio:  0.01191 seg(s)


### For Polynomial Classifier (WIP)

#### Get thresholds and the rejected for Poly

In [34]:
#T1,T2, positive_indexes,negative_indexes,rejected_indexes = find_thresholds_and_indexes(poly,X_train)
#T1,T2, positive_indexes.shape[0],negative_indexes.shape[0],rejected_indexes.shape[0]

#### Get Z3's equivalent to Poly classifier's decision function

In [35]:
#z3_dual_coef,z3_support_vectors,z3_intercept_,z3_X_Train,z3_gamma,z3_coef0,z3_degree = to_z3_conversion(poly,X_train)

In [36]:
#poly_relevant, poly_irrelevant = z3_explanation(poly,T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,rejected_indexes[0:1], z3_gamma,z3_coef0,z3_degree, show_values=False)

# Anchors

## Setting Up

In [37]:
from __future__ import print_function
import sys
import sklearn
import sklearn.ensemble
from anchor import utils
from anchor import anchor_tabular

In [38]:
def generate_ro_target_set(target_set,rejected_indexes):
    target_set[rejected_indexes] = 0
    return target_set

In [39]:
ro_set = generate_ro_target_set(y_train,rejected_indexes)
print(np.unique(ro_set))

[-1  0  1]


In [40]:
feature_list = []
for i in range(0,len(X_train[0])):
    feature_list.append(str(i))
feature_list = np.array(feature_list)

In [41]:
explainer = anchor_tabular.AnchorTabularExplainer(
    [-1,0,1],
    feature_list,
    X_train)

In [42]:
def svm_decfun(data,classifier=clf):
    data = np.atleast_2d(data)
    return ((classifier.dual_coef_ @ classifier.support_vectors_) @ data.T + classifier.intercept_)[0][0]

In [43]:
def svm_decfun_class(data,classifier=clf,Threshold_1=T1,Threshold_2=T2):
    if svm_decfun(data) >= Threshold_1:
        return np.array([2]) #class 1, since [-1, 0, 1]
    elif svm_decfun(data) < Threshold_2:
        return np.array([0]) #class -1
    else:
        return np.array([1]) #class 0

## Explanation for 1 Instance

### Anchors Explanation

In [44]:
'''idx = rejected_indexes[0]
np.random.seed(1)
print('Prediction: ', explainer.class_names[svm_decfun_class(X_train[idx])[0]])
exp = explainer.explain_instance(X_train[idx], svm_decfun_class, threshold=1)'''

"idx = rejected_indexes[0]\nnp.random.seed(1)\nprint('Prediction: ', explainer.class_names[svm_decfun_class(X_train[idx])[0]])\nexp = explainer.explain_instance(X_train[idx], svm_decfun_class, threshold=1)"

In [45]:
def explain_instance_anchors(explainer,svm_decfun_class,X_train,indexes):
    np.random.seed(1)
    elapsed_time = []
    exp = None
    for idx in indexes:    
        print('Prediction: ', explainer.class_names[svm_decfun_class(X_train[idx])[0]])
        start = time.perf_counter()  
        exp = explainer.explain_instance(X_train[idx], svm_decfun_class, threshold=1)
        elapsed_time.append(time.perf_counter()-start)
    return exp,elapsed_time

In [46]:
'''rejected_exp = None
rejected_anchors_time = None
if len(rejected_indexes)!=0:
    if len(rejected_indexes)>=50:
        rejected_exp,rejected_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,rejected_indexes[0:50])
    else:
        rejected_exp,rejected_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,rejected_indexes)
if rejected_anchors_time != None and rejected_exp != None:    
    print("Tempo Médio Anchors Rejeitados = ",sum(rejected_anchors_time)/len(rejected_anchors_time))'''

'rejected_exp = None\nrejected_anchors_time = None\nif len(rejected_indexes)!=0:\n    if len(rejected_indexes)>=50:\n        rejected_exp,rejected_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,rejected_indexes[0:50])\n    else:\n        rejected_exp,rejected_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,rejected_indexes)\nif rejected_anchors_time != None and rejected_exp != None:    \n    print("Tempo Médio Anchors Rejeitados = ",sum(rejected_anchors_time)/len(rejected_anchors_time))'

In [47]:
'''positive_exp = None
positive_anchors_time = None
if len(positive_indexes)!=0:
    if len(positive_indexes)>=50:
        positive_exp,positive_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,positive_indexes[0:50])
    else:
        positive_exp,positive_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,positive_indexes)
if positive_anchors_time != None and positive_exp != None:        
    print("Tempo Médio Anchors Positivos = ",sum(positive_anchors_time)/len(positive_anchors_time))'''

'positive_exp = None\npositive_anchors_time = None\nif len(positive_indexes)!=0:\n    if len(positive_indexes)>=50:\n        positive_exp,positive_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,positive_indexes[0:50])\n    else:\n        positive_exp,positive_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,positive_indexes)\nif positive_anchors_time != None and positive_exp != None:        \n    print("Tempo Médio Anchors Positivos = ",sum(positive_anchors_time)/len(positive_anchors_time))'

In [48]:
'''negative_exp = None
negative_anchors_time = None
if len(negative_indexes)!=0:
    if len(negative_indexes)>=50:
        negative_exp,negative_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,negative_indexes[0:50])
    else:
        negative_exp,negative_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,negative_indexes)
if negative_anchors_time != None and negative_exp != None:
    print("Tempo Médio Anchors Negativos = ",sum(negative_anchors_time)/len(negative_anchors_time))'''

'negative_exp = None\nnegative_anchors_time = None\nif len(negative_indexes)!=0:\n    if len(negative_indexes)>=50:\n        negative_exp,negative_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,negative_indexes[0:50])\n    else:\n        negative_exp,negative_anchors_time = explain_instance_anchors(explainer,svm_decfun_class,X_train,negative_indexes)\nif negative_anchors_time != None and negative_exp != None:\n    print("Tempo Médio Anchors Negativos = ",sum(negative_anchors_time)/len(negative_anchors_time))'

In [49]:
'''soma = 0
tamanho = 0
if rejected_anchors_time != None:
    soma += sum(rejected_anchors_time)
    tamanho += len(rejected_anchors_time)
    print("Tempo Médio Anchors Rejeitados = ",sum(rejected_anchors_time)/len(rejected_anchors_time))
if positive_anchors_time != None:
    soma += sum(positive_anchors_time)
    tamanho += len(positive_anchors_time)
    print("Tempo Médio Anchors Positivos = ",sum(positive_anchors_time)/len(positive_anchors_time))
if negative_anchors_time != None:
    soma += sum(negative_anchors_time)
    tamanho += len(negative_anchors_time)
    print("Tempo Médio Anchors Negativos = ",sum(negative_anchors_time)/len(negative_anchors_time))      
if tamanho != 0:
    print("Tamanho Médio Total: ",soma/tamanho,'sec')
    print("Soma: ",soma," Tamanho: ",tamanho)'''

'soma = 0\ntamanho = 0\nif rejected_anchors_time != None:\n    soma += sum(rejected_anchors_time)\n    tamanho += len(rejected_anchors_time)\n    print("Tempo Médio Anchors Rejeitados = ",sum(rejected_anchors_time)/len(rejected_anchors_time))\nif positive_anchors_time != None:\n    soma += sum(positive_anchors_time)\n    tamanho += len(positive_anchors_time)\n    print("Tempo Médio Anchors Positivos = ",sum(positive_anchors_time)/len(positive_anchors_time))\nif negative_anchors_time != None:\n    soma += sum(negative_anchors_time)\n    tamanho += len(negative_anchors_time)\n    print("Tempo Médio Anchors Negativos = ",sum(negative_anchors_time)/len(negative_anchors_time))      \nif tamanho != 0:\n    print("Tamanho Médio Total: ",soma/tamanho,\'sec\')\n    print("Soma: ",soma," Tamanho: ",tamanho)'

In [50]:
#exp.show_in_notebook()

### Anchors Explanation on Z3 - WIP

In [51]:
def anchors_to_z3_explanation(explainer, X, t1, t2, z3_coef, z3_sup_vec, z3_X, z3_intercept, indexes,
                              z3_gamma=None, z3_coef0=None, z3_degree=None,min=0,max=1,positive=False,negative=False,rejected=False):
    print('Started')
    sat_var = 0
    unsat_var = 0
    np.random.seed(1)
    solver = Solver()
    feature_sizes = []
    anchors_time = []
    if rejected:
        print("Declared: Rejected Instances")
        solver.add(Or(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t1,
                      ((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t2))
    elif positive:
        print("Declared: Positive Instances")
        solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t1)
    elif negative:
        print("Declared: Negative Instances")
        solver.add(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t2)
    else:
        print("WARNING: Must declare if are positive,negative or rejected instances!")
        return None
    for j in range(0, len(z3_X)):
        solver.add(z3_X[j] >= min)
        solver.add(z3_X[j] <= max)
    solver.push()
    for idx in indexes:
        print('Prediction: ', explainer.class_names[svm_decfun_class(X[idx])[0]])
        start = time.perf_counter()  
        exp = explainer.explain_instance(X[idx], svm_decfun_class, threshold=1)
        anchors_time.append(time.perf_counter() - start)
        #print(exp.names())
        print('Index = ', idx)
        feature_sizes.append(len(np.unique((exp.features()))))
        for feature in np.unique(exp.features()):       
            solver.add(z3_X[feature] == X[idx][feature])
            #print("Added restrictions: ",z3_X[feature],X[idx][feature])
        
        print(solver.check(), ' for ', idx,'\n')
        if solver.check() == sat:
            model = solver.model()
            print(model)
            sat_var +=1
        else:
            unsat_var +=1
        
        print('\n---------------\n')
        solver.pop()
        solver.push()
    print("Sat = ",sat_var,"\nUnsat = ",unsat_var)
    return sat_var,unsat_var,feature_sizes,anchors_time

In [52]:
rejected_sat_var = 0
rejected_unsat_var = 0
positive_sat_var = 0
positive_unsat_var = 0
negative_sat_var = 0
negative_unsat_var = 0
rejected_feature_sizes = []
positive_feature_sizes = []
negative_feature_sizes = []
r_time = []
p_time = []
n_time  = []

In [53]:
if len(rejected_indexes)!=0:
    if len(rejected_indexes)>=50:
        rejected_sat_var, rejected_unsat_var, rejected_feature_sizes,r_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,rejected_indexes[0:50],rejected=True)
    else:
        rejected_sat_var, rejected_unsat_var, rejected_feature_sizes,r_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,rejected_indexes,rejected=True)

Started
Declared: Rejected Instances
Prediction:  0
Index =  16
sat  for  16 

[x__4 = 29725514474334327/500000000000000000,
 x__9 = 7396405674248603/2500000000000000000,
 x__11 = 9941245453517237/5000000000000000000,
 x__6 = 424139963211439/250000000000000000,
 x__5 = 1,
 x__8 = 5149207974426593/5000000000000000000,
 x__0 = 1,
 x__2 = 1,
 x__3 = 378935572395483/31250000000000000,
 x__10 = 1083417169185711/2000000000000000000,
 x__1 = 5000386934703281/5000000000000000000,
 x__12 = 39711730388008691351530236527992537543181858161743237883/51292864442596728074729477997391600000000000000000000000,
 x__7 = 386934703280611/5000000000000000000]

---------------

Prediction:  0
Index =  63
unsat  for  63 


---------------

Sat =  1 
Unsat =  1


In [54]:
if len(positive_indexes)!=0:
    if len(positive_indexes)>=50:
        positive_sat_var, positive_unsat_var, positive_feature_sizes,p_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,positive_indexes[0:50],positive=True)
    else:
        positive_sat_var, positive_unsat_var, positive_feature_sizes,p_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,positive_indexes,positive=True)

Started
Declared: Positive Instances
Prediction:  1
Index =  1
sat  for  1 

[x__4 = 1,
 x__9 = 0,
 x__11 = 1631078595367499/1000000000000000000,
 x__6 = 1,
 x__5 = 1,
 x__8 = 1,
 x__0 = 1,
 x__2 = 1,
 x__3 = 0,
 x__10 = 1,
 x__1 = 4107460696363409/5000000000000000000,
 x__12 = 1815618212165834468511385945415557105805301130825685623/2564643222129836403736473899869580000000000000000000000,
 x__7 = 0]

---------------

Prediction:  1
Index =  2
sat  for  2 

[x__4 = 1,
 x__9 = 0,
 x__11 = 1631078595367499/1000000000000000000,
 x__6 = 1,
 x__5 = 1,
 x__8 = 1,
 x__0 = 761963723383357/100000000000000000,
 x__2 = 1,
 x__3 = 0,
 x__10 = 1,
 x__1 = 4107460696363409/5000000000000000000,
 x__12 = 45533716238994454998059717712869890653511910778899484571/64116080553245910093411847496739500000000000000000000000,
 x__7 = 2976420794466239/12500000000000000000]

---------------

Prediction:  1
Index =  3
sat  for  3 

[x__4 = 1,
 x__9 = 0,
 x__11 = 1631078595367499/1000000000000000000,
 x__6 = 1,
 x__

In [55]:
if len(negative_indexes)!=0:
    if len(negative_indexes)>=50:
        negative_sat_var, negative_unsat_var, negative_feature_sizes,n_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,negative_indexes[0:50],negative=True)
    else:
        negative_sat_var, negative_unsat_var, negative_feature_sizes,n_time = anchors_to_z3_explanation(explainer, X_train, T1, T2, z3_dual_coef, z3_support_vectors, z3_X_Train, z3_intercept_,negative_indexes,negative=True)

Started
Declared: Negative Instances
Prediction:  -1
Index =  0
unsat  for  0 


---------------

Prediction:  -1
Index =  4
unsat  for  4 


---------------

Prediction:  -1
Index =  5
unsat  for  5 


---------------

Prediction:  -1
Index =  6
unsat  for  6 


---------------

Prediction:  -1
Index =  12
unsat  for  12 


---------------

Prediction:  -1
Index =  13
unsat  for  13 


---------------

Prediction:  -1
Index =  20
unsat  for  20 


---------------

Prediction:  -1
Index =  21
unsat  for  21 


---------------

Prediction:  -1
Index =  30
unsat  for  30 


---------------

Prediction:  -1
Index =  32
unsat  for  32 


---------------

Prediction:  -1
Index =  36
unsat  for  36 


---------------

Prediction:  -1
Index =  39
unsat  for  39 


---------------

Prediction:  -1
Index =  40
unsat  for  40 


---------------

Prediction:  -1
Index =  49
unsat  for  49 


---------------

Prediction:  -1
Index =  54
unsat  for  54 


---------------

Prediction:  -1
Index =  5

## Calculating Metrics

### Z3 AND ANCHORS TIME COST

In [56]:
soma = 0
tamanho = 0
if rejected_elapsed_time != None and len(rejected_elapsed_time) >0:
    soma += sum(rejected_elapsed_time)
    tamanho += len(rejected_elapsed_time)
    print("Tempo Médio Z3 Rejeitados = ",sum(rejected_elapsed_time)/len(rejected_elapsed_time))
if positive_elapsed_time != None and len(positive_elapsed_time) >0:
    soma += sum(positive_elapsed_time)
    tamanho += len(positive_elapsed_time)
    print("Tempo Médio Z3 Positivos = ",sum(positive_elapsed_time)/len(positive_elapsed_time))
if negative_elapsed_time != None and len(negative_elapsed_time) >0:
    soma += sum(negative_elapsed_time)
    tamanho += len(negative_elapsed_time)
    print("Tempo Médio Z3 Negativos = ",sum(negative_elapsed_time)/len(negative_elapsed_time)) 
if tamanho != 0:
    print("Tamanho Médio Total: ",soma/tamanho,'sec')
    print("Soma: ",soma," Tamanho: ",tamanho)

Tempo Médio Z3 Rejeitados =  0.021019748500293645
Tempo Médio Z3 Positivos =  0.011129136280051171
Tempo Médio Z3 Negativos =  0.011913763531282484
Tamanho Médio Total:  0.011663532666716492 sec
Soma:  0.9797367440041853  Tamanho:  84


In [57]:
soma = 0
tamanho = 0
if r_time != None and len(r_time)>0:
    soma += sum(r_time)
    tamanho += len(r_time)
    print("Tempo Médio Anchors Rejeitados = ",sum(r_time)/len(r_time))
if p_time != None and len(p_time)>0:
    soma += sum(p_time)
    tamanho += len(p_time)
    print("Tempo Médio Anchors Positivos = ",sum(p_time)/len(p_time))
if n_time != None and len(n_time)>0:
    soma += sum(n_time)
    tamanho += len(n_time)
    print("Tempo Médio Anchors Negativos = ",sum(n_time)/len(n_time))      
if tamanho != 0:
    print("Tamanho Médio Total: ",soma/tamanho,'sec')
    print("Soma: ",soma," Tamanho: ",tamanho)

Tempo Médio Anchors Rejeitados =  1.3734633805001977
Tempo Médio Anchors Positivos =  0.4563983116399322
Tempo Médio Anchors Negativos =  0.5025907896250317
Tamanho Médio Total:  0.4958303287023574 sec
Soma:  41.64974761099802  Tamanho:  84


### Z3 AND ANCHORS EXPLANATION SIZE

In [58]:
soma = 0
tamanho = 0
if len(rejected_linear_relevant)>0:  
    tamanho += len(rejected_linear_relevant)
    tamanho_rejeitado = 0
    for x in rejected_linear_relevant:
        soma += len(x)
        tamanho_rejeitado += len(x)
    print("Z3 Tamanho Rejeitado: ",tamanho_rejeitado,"Tamanho Médio Rejeitado:",tamanho_rejeitado/len(rejected_linear_relevant))
if len(positive_linear_relevant)>0:   
    tamanho += len(positive_linear_relevant)
    tamanho_positivo = 0
    for x in positive_linear_relevant:
        soma += len(x)
        tamanho_positivo += len(x)
    print("Z3 Tamanho Positivo: ",tamanho_positivo,"Tamanho Médio Positivo:",tamanho_positivo/len(positive_linear_relevant))
if len(negative_linear_relevant)>0:
    tamanho += len(negative_linear_relevant)
    tamanho_negativo = 0
    for x in negative_linear_relevant:
        soma += len(x)
        tamanho_negativo += len(x)
    print("Z3 Tamanho Negativo: ",tamanho_negativo,"Tamanho Médio Negativo:",tamanho_negativo/len(negative_linear_relevant))
print("Z3 Tamanho Médio ",soma/tamanho)
print("Soma: ",soma,"Tamanho: ",tamanho)

Z3 Tamanho Rejeitado:  11 Tamanho Médio Rejeitado: 5.5
Z3 Tamanho Positivo:  70 Tamanho Médio Positivo: 1.4
Z3 Tamanho Negativo:  36 Tamanho Médio Negativo: 1.125
Z3 Tamanho Médio  1.3928571428571428
Soma:  117 Tamanho:  84


In [59]:
if len(rejected_feature_sizes) > 0 :
    print("Anchors Tamanho Rejeitado: ",sum(rejected_feature_sizes),"Tamanho Médio Rejeitado",sum(rejected_feature_sizes)/len(rejected_feature_sizes))
if len(positive_feature_sizes) > 0 :
    print("Anchors Tamanho Positivo: ",sum(positive_feature_sizes),"Tamanho Médio Positivo",sum(positive_feature_sizes)/len(positive_feature_sizes))
if len(negative_feature_sizes) > 0 :
    print("Anchors Tamanho Negativo: ",sum(negative_feature_sizes),"Tamanho Médio Negativo",sum(negative_feature_sizes)/len(negative_feature_sizes))
soma = sum(rejected_feature_sizes)+sum(positive_feature_sizes)+sum(negative_feature_sizes)
soma_len = len(rejected_feature_sizes) + len(positive_feature_sizes) + len(negative_feature_sizes)
print("Anchors Tamanho Médio: ",soma/soma_len)
print("Soma: ",soma,"Tamanho: ",soma_len)

Anchors Tamanho Rejeitado:  22 Tamanho Médio Rejeitado 11.0
Anchors Tamanho Positivo:  116 Tamanho Médio Positivo 2.32
Anchors Tamanho Negativo:  75 Tamanho Médio Negativo 2.34375
Anchors Tamanho Médio:  2.5357142857142856
Soma:  213 Tamanho:  84


### ANCHORS TO Z3 SAT AND UNSAT

In [60]:
print("          SAT/UNSAT")
print("Rejected: ",rejected_sat_var,"|",rejected_unsat_var)
print("Positive: ",positive_sat_var,"|",positive_unsat_var)
print("Negative: ",negative_sat_var,"|",negative_unsat_var)
soma_sat = rejected_sat_var + positive_sat_var + negative_sat_var
soma_unsat = rejected_unsat_var + positive_unsat_var + negative_unsat_var
print("Total:    ",soma_sat,"|",soma_unsat," of ",soma_sat+soma_unsat)

          SAT/UNSAT
Rejected:  1 | 1
Positive:  32 | 18
Negative:  0 | 32
Total:     33 | 51  of  84
